In [1]:
from PIL import Image
from PIL import Image
import numpy as np
import io
from PIL import Image, ImageCms
import math

In [ ]:
# TIFF to PNG

def convert_tiff_to_png(tiff_path, png_path):
    img = Image.open(tiff_path)
    img.save(png_path, format='PNG')
    print(f"{tiff_path}가 {png_path}(으)로 변환되었습니다.")

# 사용 예시
tiff_path = "D:\\Stain_normalization\\0129_data\\gt450\\gt450_resize1_8acc.tif"
png_path = "D:\\Stain_normalization\\0129_data\\gt450\\gt450_resize1_8acc.png"
patch_size = 256

# TIFF를 PNG로 변환
slices = convert_tiff_to_png(tiff_path, png_path)

In [ ]:
# Resizing Image

def resize_image_keep_aspect_ratio(input_path, output_path, target_size=(9309, 4998)):
    with Image.open(input_path) as img:
        # 원본 이미지의 비율 계산
        original_ratio = img.width / img.height
        target_ratio = target_size[0] / target_size[1]

        if original_ratio > target_ratio:
            # 원본 이미지의 너비가 목표 비율보다 클 경우, 높이 기준으로 리사이징
            new_height = target_size[1]
            new_width = int(new_height * original_ratio)
        else:
            # 원본 이미지의 높이가 목표 비율보다 클 경우, 너비 기준으로 리사이징
            new_width = target_size[0]
            new_height = int(new_width / original_ratio)
        
        resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # 목표 해상도 중앙에 맞춰 이미지를 크롭
        left = (new_width - target_size[0]) / 2
        top = (new_height - target_size[1]) / 2
        right = (new_width + target_size[0]) / 2
        bottom = (new_height + target_size[1]) / 2

        cropped_img = resized_img.crop((left, top, right, bottom))

        # 리사이징 및 크롭된 이미지 저장

        if "icc_profile" in cropped_img.info:
            icc_profile = cropped_img.info["icc_profile"]
            src_profile = ImageCms.ImageCmsProfile(io.BytesIO(icc_profile))
            dst_profile = ImageCms.createProfile("sRGB")
            cropped_img = ImageCms.profileToProfile(cropped_img, src_profile, dst_profile)
        cropped_img.save(output_path)

# 원본 이미지 경로와 결과 이미지 경로 설정
input_path = "D:\\Stain_normalization\\0402\\0521_newdataset\\6917f7d6-1b21-01f6-af6f-8a8fd9fd793f_143139.tif"  # 원본 이미지 경로
output_path = "D:\\Stain_normalization\\0402\\0521_newdataset\\gt450.tif"  # 저장할 이미지 경로

# 함수 호출로 이미지 리사이징 실행
resize_image_keep_aspect_ratio(input_path, output_path)

In [ ]:
# 고정된 width로 이미지 resize (해상도 낮게) 

def resize_image(input_path, output_path, target_width=8565):
    with Image.open(input_path) as img:
        original_width, original_height = img.size

        # 새로운 너비에 따른 비율 유지하여 높이 계산
        aspect_ratio = original_height / original_width
        new_width = target_width
        new_height = int(new_width * aspect_ratio)

        # 이미지 리사이즈
        resized_img = img.resize((new_width, new_height), Image.LANCZOS)

        # ICC 프로파일 적용 (있는 경우)
        if "icc_profile" in img.info:
            icc_profile = img.info["icc_profile"]
            src_profile = ImageCms.ImageCmsProfile(io.BytesIO(icc_profile))
            dst_profile = ImageCms.createProfile("sRGB")
            resized_img = ImageCms.profileToProfile(resized_img, src_profile, dst_profile)

        # 최종 이미지 저장
        resized_img.save(output_path)

resize_image("D:\\Stain_normalization\\123\\crop_gt.tif", "D:\\crop_gt.tif")

In [ ]:
# image resize half

def resize_image_half(input_path, output_path):
    with Image.open(input_path) as img:
        # 원본 이미지의 새로운 너비와 높이를 절반으로 설정
        new_width = img.width // 2
        new_height = img.height // 2
        
        # 이미지 리사이즈
        resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # ICC 프로파일 유지
        if "icc_profile" in resized_img.info:
            icc_profile = resized_img.info["icc_profile"]
            src_profile = ImageCms.ImageCmsProfile(io.BytesIO(icc_profile))
            dst_profile = ImageCms.createProfile("sRGB")
            resized_img = ImageCms.profileToProfile(resized_img, src_profile, dst_profile)
        
        # 리사이즈된 이미지 저장
        resized_img.save(output_path)

# 원본 이미지 경로와 결과 이미지 경로 설정
input_path = "D:\\Stain_normalization\\12\\6917f7d6-1b21-01f6-af6f-8a8fd9fd793f_143139_001.tif"  # 원본 이미지 경로
output_path = "D:\\Stain_normalization\\12\\gt450.tif"  # 저장할 이미지 경로

# 함수 호출로 이미지 리사이징 실행
resize_image_half(input_path, output_path)

In [ ]:
# image crop (left, top, right, bottom)

def crop_image(input_path, output_path):
    with Image.open(input_path) as img:
        left_crop = 0
        top_crop = 3
        right_crop = 16020
        bottom_crop = 8865
        #가로 시작점, 세로 시작점, 가로 범위, 세로 범위
        cropped_img = img.crop((left_crop, top_crop, right_crop, bottom_crop))

        # 리사이징 및 크롭된 이미지 저장

        if "icc_profile" in cropped_img.info:
            icc_profile = cropped_img.info["icc_profile"]
            src_profile = ImageCms.ImageCmsProfile(io.BytesIO(icc_profile))
            dst_profile = ImageCms.createProfile("sRGB")
            cropped_img = ImageCms.profileToProfile(cropped_img, src_profile, dst_profile)
        cropped_img.save(output_path)

In [ ]:
# image 회전 (양수 오른쪽 회전, 음수 왼쪽 회전)

def crop_and_rotate_image(input_path, output_path, angle=-1):
    with Image.open(input_path) as img:
        original_width, original_height = img.size
       
        # 이미지 회전 (각도를 양수로 설정하여 오른쪽으로 회전)
        rotated_img = img.rotate(angle, expand=True)

        # 회전 후 이미지의 크기
        rotated_width, rotated_height = rotated_img.size

        # 회전 후 사용할 수 있는 최대 사각형 크기 계산
        angle_rad = math.radians(abs(angle))
        cos_angle = math.cos(angle_rad)
        sin_angle = math.sin(angle_rad)

        # 계산된 너비와 높이
        max_width = int(original_width * cos_angle + original_height * sin_angle)
        max_height = int(original_height * cos_angle + original_width * sin_angle)

        # 중앙 부분 크롭 범위 계산
        left = (rotated_width - max_width) // 2
        top = (rotated_height - max_height) // 2
        right = left + max_width
        bottom = top + max_height

        # 중앙 부분 크롭
        final_img = rotated_img.crop((left, top, right, bottom))

        # ICC 프로파일 적용 (있는 경우)
        if "icc_profile" in img.info:
            icc_profile = img.info["icc_profile"]
            src_profile = ImageCms.ImageCmsProfile(io.BytesIO(icc_profile))
            dst_profile = ImageCms.createProfile("sRGB")
            final_img = ImageCms.profileToProfile(final_img, src_profile, dst_profile)

        # 최종 이미지 저장
        final_img.save(output_path)